In [ ]:
"""
You need to install these modules before you can use them
pip install <module>


This file is were i am experimenting with landmark extraction. i use open cv to read video from files and run the frames through media pipe to extract the landmark data.
"""
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
mpHands = mp.solutions.hands
mpPose = mp.solutions.pose
# mpFace = mp.solutions.
mpDraw = mp.solutions.drawing_utils


In [ ]:
# sets default configurations for the hand and pose detection
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7)
pose = mpPose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


In [ ]:
#Reading From a video file
cap = cv2.VideoCapture('videos/SamplePushups.mp4')

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
cv2.resizeWindow('MediaPipe', 960, 540)  # width, height

while cap.isOpened():
    _, frame = cap.read()

    try:
        #convert to RGB format
        RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # generate landmark data
        results = pose.process(RGB)
        print(results.pose_landmarks)
        #drawwing landmarks
        mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe', frame)
    except:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
"""
* This uses the pose model from media pipe on live video feed to track landmarks
"""
#Reading From a live camera feed.
cap = cv2.VideoCapture(0)

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('MediaPipe', 960, 540)  # width, height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)  # width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)  # height
cap.set(cv2.CAP_PROP_FPS, 30)

while True:
    _, frame = cap.read()
    x, y, c = frame.shape
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (640, 360))
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    poseResults = pose.process(framergb)

    mpDraw.draw_landmarks(frame, poseResults.pose_landmarks, mpPose.POSE_CONNECTIONS)

    cv2.imshow('MediaPipe', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
"""
* Reads from live video feed to track landmarks for hand data
"""
#Reading From a video file
cap = cv2.VideoCapture(0)

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('MediaPipe', 960, 540)  # width, height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)  # width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)  # height
cap.set(cv2.CAP_PROP_FPS, 30)


while True:
    _, frame = cap.read()
    x, y, c = frame.shape
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (640, 360))
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(framergb)
    if results.multi_hand_landmarks:
        landmarks = []
        for handslms in results.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)
                landmarks.append([lmx, lmy])
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

    cv2.imshow('MediaPipe', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
"""
* Reads from live video feed to track landmarks for hand and pose data
* Press E - to show all landmarks
* Press P - to show pose data
* Press Q - to exit
* Press H - to show hand data
* Press R - to hide all landmarks
"""
#Reading From a video file
cap = cv2.VideoCapture(0)

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('MediaPipe', 960, 540)  # width, height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # height
cap.set(cv2.CAP_PROP_FPS, 30)

showHands = True
showPose = True

while True:
    _, frame = cap.read()
    x, y, c = frame.shape

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (640, 360))
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if showHands:
        results = hands.process(framergb)
    if showPose:
        poseResults = pose.process(framergb)
    if showHands:
        if results.multi_hand_landmarks:
            landmarks = []
            for handslms in results.multi_hand_landmarks:
                for lm in handslms.landmark:
                    lmx = int(lm.x * x)
                    lmy = int(lm.y * y)
                    landmarks.append([lmx, lmy])
                mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
    if showPose:
        mpDraw.draw_landmarks(frame, poseResults.pose_landmarks, mpPose.POSE_CONNECTIONS)

    cv2.imshow('MediaPipe', frame)
    keybind = cv2.waitKey(1) & 0xFF
    if keybind == ord('q'):
        break
    if keybind == ord('p'):
        showPose = not showPose
    if keybind == ord('h'):
        showHands = not showHands
    if keybind == ord('e'):
        showPose = True
        showHands = True
    if keybind == ord('r'):
        showPose = False
        showHands = False
cap.release()
cv2.destroyAllWindows()
